In [1]:
# file: get_balance_sheets.py

import time
import pandas as pd
import tushare as ts
from sqlalchemy import create_engine
from tqdm import tqdm

# 导入您的配置文件
try:
    import config
except ImportError:
    print("错误：无法找到配置文件 config.py。")
    exit()

# --- 资产负债表数据获取配置 ---
BATCH_SIZE = 100  # 每处理100家公司的数据就保存一次
BALANCE_SHEET_TABLE_NAME = 'stock_balance_sheets' # 储存资产负债表的新表名

START_DATE = '20190101'
END_DATE = '20241231'
# --- 明确指定需要从资产负债表中获取的字段 ---
# 我们把最重要的 total_assets (总资产) 放在前面
BALANCE_SHEET_FIELDS = [
    'ts_code', 'ann_date', 'end_date', 'report_type', 'comp_type',
    'total_assets',       # 总资产
    'total_liab',         # 总负债
    'total_hldr_eqy_inc_min_int', # 股东权益合计(含少数股东权益)
    'cap_rese',           # 资本公积金
    'undistr_porfit',     # 未分配利润
    'accounts_receiv',    # 应收账款
    'inventory',          # 存货
    # 您可以根据Tushare文档，按需添加更多资产负债表字段
]

def save_batch_to_db(data_list, db_engine, table_name):
    """将一个批次的数据保存到数据库的指定表中"""
    if not data_list:
        return
    try:
        batch_df = pd.concat(data_list, ignore_index=True)
        batch_df.to_sql(table_name, db_engine, if_exists='append', index=False)
    except Exception as e:
        print(f"\n保存批次数据到表 '{table_name}' 时出错: {e}")

def fetch_all_balance_sheet_data():
    """
    (可断点续传版)
    获取所有A股公司的资产负债表数据并存入新表。
    """
    # 1. 初始化和连接
    try:
        pro = ts.pro_api(config.TUSHARE_TOKEN)
        db_uri = (
            f"postgresql+psycopg2://{config.DB_USER}:{config.DB_PASS}@"
            f"{config.DB_HOST}:{config.DB_PORT}/{config.DB_NAME}"
        )
        engine = create_engine(db_uri)
        print("Tushare API 和数据库连接初始化成功。")
    except Exception as e:
        print(f"初始化连接时发生错误: {e}")
        return

    # 2. 断点续传逻辑: 检查新表中已处理过的股票
    processed_codes = set()
    try:
        with engine.connect() as connection:
            if engine.dialect.has_table(connection, BALANCE_SHEET_TABLE_NAME):
                processed_df = pd.read_sql(f"SELECT DISTINCT ts_code FROM {BALANCE_SHEET_TABLE_NAME}", engine)
                processed_codes = set(processed_df['ts_code'])
                print(f"检测到目标表 '{BALANCE_SHEET_TABLE_NAME}'，已处理过 {len(processed_codes)} 只股票。")
    except Exception as e:
        print(f"检查已处理股票时出错（如果表是首次创建，此信息可忽略）: {e}")

    # 3. 获取全部待处理的股票列表
    try:
        all_stocks_df = pd.read_sql("SELECT ts_code FROM stock_basic_info", engine)
        full_stock_list = all_stocks_df['ts_code'].tolist()
        codes_to_process = [code for code in full_stock_list if code not in processed_codes]
        if not codes_to_process:
            print("所有股票的资产负债表数据均已获取，无需执行任务。")
            return
        print(f"共 {len(full_stock_list)} 只股票，还需处理 {len(codes_to_process)} 只。")
    except Exception as e:
        print(f"从数据库读取股票列表时出错: {e}")
        return

    # 4. 批处理循环获取数据
    batch_data = []
    print(f"开始获取资产负债表数据，将存入表 '{BALANCE_SHEET_TABLE_NAME}'...")
    print(f"每次请求的字段: {BALANCE_SHEET_FIELDS}")

    for ts_code in tqdm(codes_to_process, desc="正在获取资产负债表"):
        try:
            # 使用 pro.balancesheet 接口
            df = pro.balancesheet(
                ts_code=ts_code,
                fields=','.join(BALANCE_SHEET_FIELDS)
            )
            if not df.empty:
                batch_data.append(df)
            
            if len(batch_data) >= BATCH_SIZE:
                save_batch_to_db(batch_data, engine, BALANCE_SHEET_TABLE_NAME)
                batch_data = []

            time.sleep(0.6) # Tushare资产负债表接口每分钟200次，0.6s间隔较安全

        except Exception as e:
            print(f"\n获取 {ts_code} 数据时出错: {e}。跳过该股票。")
            continue

    # 5. 处理并保存最后一个批次
    if batch_data:
        print("\n正在保存最后一个批次的数据...")
        save_batch_to_db(batch_data, engine, BALANCE_SHEET_TABLE_NAME)

    print(f"\n所有剩余股票的资产负债表数据处理完毕！数据已存入 '{BALANCE_SHEET_TABLE_NAME}' 表。")

if __name__ == '__main__':
    fetch_all_balance_sheet_data()

Tushare API 和数据库连接初始化成功。
共 5413 只股票，还需处理 5413 只。
开始获取资产负债表数据，将存入表 'stock_balance_sheets'...
每次请求的字段: ['ts_code', 'ann_date', 'end_date', 'report_type', 'comp_type', 'total_assets', 'total_liab', 'total_hldr_eqy_inc_min_int', 'cap_rese', 'undistr_porfit', 'accounts_receiv', 'inventory']


正在获取资产负债表:   9%|█▋                | 499/5413 [06:18<1:02:46,  1.30it/s]/var/folders/jk/_51b3nb118q00wt960d15lz00000gn/T/ipykernel_45637/1016402261.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  batch_df = pd.concat(data_list, ignore_index=True)
正在获取资产负债表:  66%|███████████▏     | 3557/5413 [54:32<5:00:05,  9.70s/it]


获取 600869.SH 数据时出错: HTTPConnectionPool(host='127.0.0.1', port=7890): Read timed out. (read timeout=30)。跳过该股票。


正在获取资产负债表: 100%|█████████████████| 5413/5413 [1:25:03<00:00,  1.06it/s]


正在保存最后一个批次的数据...

所有剩余股票的资产负债表数据处理完毕！数据已存入 'stock_balance_sheets' 表。
